In [32]:
import numpy as np
import random as rd
import Queue  as queue
import collections
import time

### Para este proyecto se utiliza la clase nodo para poder realizar las busquedas de forma mas sencilla

In [33]:
class Nodo:

    def __init__(self,board,pos0,parent=False):
        self.board = board
        self.posEmpty = pos0
        self.parent = parent
        self.FindNeighbours()
    
        
    def FindNeighbours(self):
        self.neighbours = []
        nboard = []

        vals = [[0,-1],[0,1],[1,0],[-1,0]]
        for i in vals:
            nboard = self.board.copy()
            band = False
            x = self.posEmpty[0] + i[0]
            y = self.posEmpty[1] + i[1]
            if x >=0 and y >=0 and x < 3 and y < 3:
                temp = self.board[x][y]
                nboard[x][y] = nboard[self.posEmpty[0]][self.posEmpty[1]]
                nboard[self.posEmpty[0]][self.posEmpty[1]] = temp
                if self.parent:
                    band = isObjetive(self.parent,nboard)
                if not band:
                    self.neighbours.append([nboard,[x,y],self])
    def status(self):
        return str(self.board)
    
    def getSequence(self):
        apc = self
        seq = []
        while apc:
            seq.append(apc)
            apc = apc.parent
        return reversed(seq)

### FindEmpty(board)
Esta funci&oacute;n se encarga de encontrar el espacio vacio en el mapa actual 

In [34]:
def FindEmpty(board):
    for row in range(0,3):
        for col in range(0,3):
            if board[row,col] == 0:
                return [row,col]

### Esta funcion genera un estado aleatorio para el 8-Puzzle

In [35]:
def Crea_Estado_Aleatorio():
    lista = range(0,9)
    lista = rd.sample(lista,k=9)
    esIn = np.array([lista[0:3],lista[3:6],lista[6:10]])
    posEmpty = FindEmpty(esIn)
    esIn = Nodo(esIn,posEmpty)

    return esIn

### Esta funcion verifica si el estado actual es el objetivo

In [36]:
def isObjetive(esAct,esObj):
    return np.array_equal(esAct.board,esObj)

### Solvable(start)
Ya que se generan estados aleatorios es necesario verificar si el estado inicial es posible de resolver <br>
Para eso se calcula la cantidad de inversiones en el estado. <br>

Entiendase una inversi&oacute;n como un par de elementos que estan invertidos en el mapa con respecto al estado objetivo, por ejemplo:<br>
![title](8puzzle.png)

En el mapa izquierdo anterior tiene 10 inversiones y el derecho tiene 11, lo que hace un mapa posible de resolver es que tenga un n&uacute;mero par de inversiones.

In [37]:

def Solvable(start):
    count = 0
    tiles = [start.board[0][0],start.board[0][1],start.board[0][2],
     start.board[1][0],start.board[1][1],start.board[1][2],
     start.board[2][0],start.board[2][1],start.board[2][2]]

    
    for i in range(9):
        for j in range(i+1,9):
            if tiles[j] >0 and tiles[i] > 0 and tiles[i] > tiles[j]:
                count += 1
    return (count%2 == 0)

### DFS(start,obj)

In [38]:
##DFS
def DFS(start,obj):
    maxnodes = 0
    node_obj = None
    explored = set()
    estado = collections.deque([start])
    explored.add(start.status())

    band = Solvable(start)
    while estado and band:
        actual = estado.pop()

        if isObjetive(actual,obj):
            print "Yeii"
            return actual.getSequence(),len(explored)
        
        for vec in actual.neighbours:

            newVec = Nodo(vec[0],vec[1],vec[2])
            
            if newVec.status() not in explored:
                estado.appendleft(newVec)
                explored.add(newVec.status())
        if len(estado) > maxnodes:
            maxnode = len(estado)
            
    return False


### BFS(start,obj)

In [39]:
def BFS(start,obj):
    maxnodes = 0
    node_obj = None
    explored = set()
    estado = collections.deque([start])
    explored.add(start.status())

    band = Solvable(start)
    while estado and band:
        actual = estado.pop()

        if isObjetive(actual,obj):
            print "Yeii"
            return actual.getSequence(),len(explored)
        
        for vec in actual.neighbours:

            newVec = Nodo(vec[0],vec[1],vec[2])
            
            if newVec.status() not in explored:
                estado.append(newVec)
                explored.add(newVec.status())

            
    return False

### Generamos un estado aleatorio y el estado objetivo

In [40]:
esObj = np.array([[1,2,3],[4,5,6],[7,8,0]])
esSt = Crea_Estado_Aleatorio()
print esSt.board

[[3 2 7]
 [8 1 5]
 [0 4 6]]


### Aplicamos DFS

In [41]:
node_obj = None
start_time = time.time()
node_obj,nodos = DFS(esSt,esObj)
print("--- %s Nodos insertados al estado ---" %nodos)
print("--- %s segundos ---" % (time.time() - start_time))
actual = node_obj
if node_obj:
    for actual in node_obj:
        print actual.board,"\n"
        actual = actual.parent
else: 
    print "Estado inicial imposible de resolver\n",esSt.board
        

Yeii
--- 69195 Nodos insertados al estado ---
--- 28.6682469845 segundos ---
[[3 2 7]
 [8 1 5]
 [0 4 6]] 

[[3 2 7]
 [0 1 5]
 [8 4 6]] 

[[3 2 7]
 [1 0 5]
 [8 4 6]] 

[[3 0 7]
 [1 2 5]
 [8 4 6]] 

[[3 7 0]
 [1 2 5]
 [8 4 6]] 

[[3 7 5]
 [1 2 0]
 [8 4 6]] 

[[3 7 5]
 [1 0 2]
 [8 4 6]] 

[[3 0 5]
 [1 7 2]
 [8 4 6]] 

[[0 3 5]
 [1 7 2]
 [8 4 6]] 

[[1 3 5]
 [0 7 2]
 [8 4 6]] 

[[1 3 5]
 [7 0 2]
 [8 4 6]] 

[[1 3 5]
 [7 2 0]
 [8 4 6]] 

[[1 3 0]
 [7 2 5]
 [8 4 6]] 

[[1 0 3]
 [7 2 5]
 [8 4 6]] 

[[1 2 3]
 [7 0 5]
 [8 4 6]] 

[[1 2 3]
 [7 4 5]
 [8 0 6]] 

[[1 2 3]
 [7 4 5]
 [0 8 6]] 

[[1 2 3]
 [0 4 5]
 [7 8 6]] 

[[1 2 3]
 [4 0 5]
 [7 8 6]] 

[[1 2 3]
 [4 5 0]
 [7 8 6]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]] 



### Aplicamos BFS

In [42]:
node_obj = None
start_time = time.time()
node_obj,nodos = BFS(esSt,esObj)
cont = 0
print("--- %s Nodos insertados al estado ---" %nodos)
print("--- %s segundos ---" % (time.time() - start_time))
actual = node_obj
if node_obj:
    for actual in node_obj:
        cont +=1
        actual = actual.parent
    print "--- %s Nodos para el resultado --- " %cont
else: 
    print "Estado inicial imposible de resolver\n",esSt.board
        

Yeii
--- 110181 Nodos insertados al estado ---
--- 42.264950037 segundos ---
--- 58497 Nodos para el resultado --- 
